<b>This is a convolutional neural network model with the following architecture</b>
<img src="network_modelV2_CNN.jpg" width="1200">

<b>Loading data</b>

In [1]:
import torch
from torch import nn 

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.FashionMNIST(root="data", train=True, transform=ToTensor(), download=True)
test_data = datasets.FashionMNIST(root="data", train=False, transform=ToTensor(), download=True)

from torch.utils.data import DataLoader
BATCH_SIZE = 32
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)

device = "cuda" if torch.cuda.is_available() else "cpu"

learning to work with Conv2d layer

In [3]:
from matplotlib import pyplot as plt

images = torch.randn(size=(32, 3, 64, 64)) # [batch_size, color_channels, height, width]
test_image = images[0] # get a single image for testing

# test_image =  test_image[2, :, :]
# plt.imshow(test_image)

conv_layer = nn.Conv2d(in_channels = 3,
                        out_channels = 10, 
                        kernel_size = (4,4),
                        stride=1, 
                        padding=0)

conv_layer_out = conv_layer(test_image)
print(conv_layer_out.shape)
conv_layer.state_dict()["weight"].shape

torch.Size([10, 61, 61])


torch.Size([10, 3, 4, 4])

learning to work with nn.MaxPool2d \
other methods of pooling are: 
* L2 pooling 

In [4]:
max_pool_layer = nn.MaxPool2d(kernel_size=(3,3))
max_pool_layer(conv_layer_out).shape

torch.Size([10, 20, 20])

<b>Making Model</b>

In [5]:
from model_classes import FashionMNISTV2    

model_2 = FashionMNISTV2(input_shape=1, hidden_units=10, output_shape=10)
model_2.to(device)

model_2.state_dict().keys()

odict_keys(['block1.0.weight', 'block1.0.bias', 'block1.2.weight', 'block1.2.bias', 'block2.0.weight', 'block2.0.bias', 'block2.2.weight', 'block2.2.bias', 'classifier.1.weight', 'classifier.1.bias'])

**Training the model**

In [6]:
#evaluating the model initialized with random weights and biases
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)

from helper_func import eval_model
eval_model(model_2, test_dataloader, loss_fn, device)

{'model_name': 'FashionMNISTV2',
 'model_acc': 9.335063897763577,
 'model_loss': tensor(2.3046, grad_fn=<DivBackward0>)}

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)


from timeit import default_timer as timer 

from helper_func import train_model
train_model(model_2, loss_fn, optimizer, train_dataloader, epochs=3, device=device)


training in epoch:  0
finished training with batch  400
finished training with batch  800
finished training with batch  1200
finished training with batch  1600
finished training of epoch  0
training in epoch:  1
finished training with batch  400
finished training with batch  800
finished training with batch  1200
finished training with batch  1600
finished training of epoch  1
training in epoch:  2
finished training with batch  400
finished training with batch  800
finished training with batch  1200
finished training with batch  1600
finished training of epoch  2
finished training


In [8]:
from helper_func import eval_model
eval_model(model_2, test_dataloader, loss_fn, device)

{'model_name': 'FashionMNISTV2',
 'model_acc': 87.80950479233228,
 'model_loss': tensor(0.3248, grad_fn=<DivBackward0>)}

<b>Saving the model </b>

In [10]:
torch.save(model_2.state_dict(), 'saved_models/model_v2.mod')